## **Создаем Spark-сессию**

In [1]:
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.window import Window

In [3]:
jar_files = [
    "/usr/local/spark/jars/postgresql-42.6.0.jar",
    "/usr/local/spark/jars/clickhouse-jdbc-0.4.6-all.jar"
]

In [4]:
spark = (
    SparkSession
    .builder
    .appName("SparkDataMart")
    .config("spark.jars", ",".join(jar_files))
    .getOrCreate()
)

25/11/09 13:15:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
",".join(jar_files)

'/usr/local/spark/jars/postgresql-42.6.0.jar,/usr/local/spark/jars/clickhouse-jdbc-0.4.6-all.jar'

In [6]:
spark

## **Подключаемся к источникам**

**csv**

In [8]:
path = "/home/jovyan/work/data"

In [9]:
campaigns_dict = (
    spark.read
    .option('header', True)
    .csv(f'{path}/campaigns_dict.csv')
)

In [10]:
# ленивые вычисления (transformations, actions)
campaigns_dict.show(5, truncate=False) #truncate=False - чтобы Spark не обрезал текст до определенного количества символов при выводе, а выводил все

+-----------+------------------------------------------+
|campaign_id|campaign_name                             |
+-----------+------------------------------------------+
|1          |year_modern_kitchen_launch_20250115       |
|2          |quarter_custom_kitchens_showcase_20240210 |
|3          |month_smart_kitchen_promotion_20240305    |
|4          |year_luxury_kitchens_exhibit_20240420     |
|5          |quarter_ecofriendly_kitchen_offer_20240512|
+-----------+------------------------------------------+
only showing top 5 rows


In [11]:
campaigns_dict.printSchema()

root
 |-- campaign_id: string (nullable = true)
 |-- campaign_name: string (nullable = true)



**parquet**

In [12]:
submits = spark.read.parquet(f'{path}/submits.parquet')

In [13]:
submits.show(5, truncate=False)

+---------+--------+-----------+
|submit_id|name    |phone      |
+---------+--------+-----------+
|2282     |Jennifer|79511904041|
|9898     |Jeffrey |79824419733|
|9005     |Linda   |79074725672|
|1507     |Teresa  |79864203598|
|3803     |Tanya   |79779567654|
+---------+--------+-----------+
only showing top 5 rows


In [14]:
submits.printSchema()

root
 |-- submit_id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- phone: long (nullable = true)



In [15]:
deals = spark.read.parquet(f'{path}/deals.parquet')

In [16]:
deals.show(5, truncate=False)

+-------+----------+---------------------+-----------+------------------------+-----------------------------------------------------+
|deal_id|deal_date |fio                  |phone      |email                   |address                                              |
+-------+----------+---------------------+-----------+------------------------+-----------------------------------------------------+
|1      |2024-03-04|Gregory Wu           |79746561889|paul80@example.net      |098 Yates Cliff Apt. 241, East Monica, DE 88076      |
|2      |2024-08-20|William Ross Jr.     |79074725672|xyoung@example.org      |197 Willie Groves Apt. 655, Port Angelaberg, LA 39384|
|3      |2024-10-15|Sonya Kerr           |79201244835|elewis@example.com      |144 Andrew Cape, Lake Nicholas, SC 58918             |
|4      |2024-12-31|Mrs. Angela Tucker MD|79771829751|robertparker@example.net|6056 Collins View, South Harold, OR 15650            |
|5      |2024-03-23|Eric Flores          |79729054809|barbara7

In [17]:
deals.show(2, truncate=False, vertical=True)

-RECORD 0----------------------------------------------------------
 deal_id   | 1                                                     
 deal_date | 2024-03-04                                            
 fio       | Gregory Wu                                            
 phone     | 79746561889                                           
 email     | paul80@example.net                                    
 address   | 098 Yates Cliff Apt. 241, East Monica, DE 88076       
-RECORD 1----------------------------------------------------------
 deal_id   | 2                                                     
 deal_date | 2024-08-20                                            
 fio       | William Ross Jr.                                      
 phone     | 79074725672                                           
 email     | xyoung@example.org                                    
 address   | 197 Willie Groves Apt. 655, Port Angelaberg, LA 39384 
only showing top 2 rows


In [18]:
deals.printSchema()

root
 |-- deal_id: long (nullable = true)
 |-- deal_date: string (nullable = true)
 |-- fio: string (nullable = true)
 |-- phone: long (nullable = true)
 |-- email: string (nullable = true)
 |-- address: string (nullable = true)



**Установка pyarrow**

In [19]:
# Удалим все старые закачки
!pip cache purge

Files removed: 6


In [20]:
# Обновим pip до последней версии
!pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 138.1 kB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 23.3
    Uninstalling pip-23.3:
      Successfully uninstalled pip-23.3


In [21]:
# Установим pyarrow без кэша и без проверки хэшей
!pip install --no-cache-dir --no-deps pyarrow

   ━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━ 20.0/47.7 MB 275.9 kB/s  0:01:41
Resuming download pyarrow-22.0.0-cp311-cp311-manylinux_2_28_x86_64.whl (20.0 MB/47.7 MB)
   ━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━ 21.8/47.7 MB 240.0 kB/s  0:01:48
Resuming download pyarrow-22.0.0-cp311-cp311-manylinux_2_28_x86_64.whl (21.8 MB/47.7 MB)
   ━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━ 23.8/47.7 MB 73.8 kB/s  0:05:25
Resuming download pyarrow-22.0.0-cp311-cp311-manylinux_2_28_x86_64.whl (23.8 MB/47.7 MB)
   ━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━ 26.7/47.7 MB 73.0 kB/s  0:04:49m
Resuming download pyarrow-22.0.0-cp311-cp311-manylinux_2_28_x86_64.whl (26.7 MB/47.7 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 195.9 kB/s  0:01:03m0:00:0200:08


In [19]:
import pyarrow as pa
import pyarrow.parquet as pq
print(pa.__version__)

22.0.0


In [20]:
pq.read_metadata(f'{path}/submits.parquet')  # Parquet recommends row groups sized between 512MB and 1GB

  created_by: parquet-cpp-arrow version 19.0.0
  num_columns: 3
  num_rows: 4000
  num_row_groups: 1
  format_version: 2.6
  serialized_size: 2371

In [21]:
submits.count() # количество строк

4000

**postgres**

In [28]:
# чтение датафрейма
df_csv = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv(f'{path}/costs_postgres.csv')

In [30]:
df_csv.show()

+----------+-----------+------+------+-----+
|      date|campaign_id| costs|clicks|views|
+----------+-----------+------+------+-----+
|2024-01-01|          1|670.52|    40|  110|
|2024-01-01|          2| 602.5|    11|  849|
|2024-01-01|          3|654.74|    51|  566|
|2024-01-01|          4|897.24|    86|  679|
|2024-01-01|          5|758.19|    30|  585|
|2024-01-01|          6|523.91|    77|  883|
|2024-01-01|          7|465.35|    98|  527|
|2024-01-01|          8|771.47|     4|  585|
|2024-01-01|          9|973.09|    51|  255|
|2024-01-01|         10|886.07|    88|  815|
|2024-01-01|         11|489.74|    54|  624|
|2024-01-01|         12|522.04|    25|  898|
|2024-01-01|         13|254.72|    23|  895|
|2024-01-01|         14| 840.0|    89|  302|
|2024-01-01|         15|420.02|    64|  974|
|2024-01-01|         16|783.93|    38|  202|
|2024-01-01|         17| 86.72|    30|  554|
|2024-01-01|         18|480.09|    41|  484|
|2024-01-01|         19|856.34|    95|  268|
|2024-01-0

In [34]:
df_csv.printSchema()

root
 |-- date: date (nullable = true)
 |-- campaign_id: integer (nullable = true)
 |-- costs: double (nullable = true)
 |-- clicks: integer (nullable = true)
 |-- views: integer (nullable = true)



In [42]:
pg_host = 'postgres'
pg_port = '5432'
pg_db = 'mydb'
pg_table = 'costs'
pg_user = 'admin'
pg_password = 'admin'

In [43]:
# Пишем в PostgreSQL — Spark сам создаст таблицу, если её нет
# overwrite чтобы создать заново
df_csv.write \
    .format("jdbc") \
    .option("url", f'jdbc:postgresql://{pg_host}:{pg_port}/{pg_db}') \
    .option("dbtable", pg_table) \
    .option("user", pg_user) \
    .option("password", pg_password) \
    .option("driver", "org.postgresql.Driver") \
    .mode("overwrite") \
    .save()

In [44]:
costs = (
    spark.read
    .format('jdbc')
    .option('url', f'jdbc:postgresql://{pg_host}:{pg_port}/{pg_db}')
    .option('dbtable', pg_table)
    .option('user', pg_user)
    .option('password', pg_password)
    .option('driver', 'org.postgresql.Driver')
    .load()
)

In [45]:
costs.show(5, truncate=False)  # заглянуть в pgAdmin

+----------+-----------+------+------+-----+
|date      |campaign_id|costs |clicks|views|
+----------+-----------+------+------+-----+
|2024-01-01|1          |670.52|40    |110  |
|2024-01-01|2          |602.5 |11    |849  |
|2024-01-01|3          |654.74|51    |566  |
|2024-01-01|4          |897.24|86    |679  |
|2024-01-01|5          |758.19|30    |585  |
+----------+-----------+------+------+-----+
only showing top 5 rows


In [46]:
costs.printSchema()

root
 |-- date: date (nullable = true)
 |-- campaign_id: integer (nullable = true)
 |-- costs: double (nullable = true)
 |-- clicks: integer (nullable = true)
 |-- views: integer (nullable = true)



**ClickHouse**

In [47]:
# чтение датафрейма
df_csv_click = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv(f'{path}/visits_clickhouse.csv')

In [49]:
df_csv_click.show(2)

+-------+-------------------+--------------------+--------+--------+--------+--------------------+----------------+
|visitid|      visitDateTime|                 URL|duration|clientID|  source|         UTMCampaign|          params|
+-------+-------------------+--------------------+--------+--------+--------+--------------------+----------------+
| 720128|2024-01-23 18:23:48|https://our-cool-...|      80|     289|      ad|month_openconcept...|              []|
| 695905|2024-07-24 03:32:19|https://our-cool-...|      66|     765|internal|month_contemporar...|['submit', 9469]|
+-------+-------------------+--------------------+--------+--------+--------+--------------------+----------------+
only showing top 2 rows


In [50]:
df_csv_click.printSchema()

root
 |-- visitid: integer (nullable = true)
 |-- visitDateTime: timestamp (nullable = true)
 |-- URL: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- clientID: integer (nullable = true)
 |-- source: string (nullable = true)
 |-- UTMCampaign: string (nullable = true)
 |-- params: string (nullable = true)



In [66]:
# Параметры ClickHouse
ch_host = 'clickhouse'
ch_port = '8123'
ch_db = 'default'
ch_table = 'visits'
ch_user = 'default'
ch_password = 'mypassword'
driver = "com.clickhouse.jdbc.ClickHouseDriver"
url = f"jdbc:clickhouse://{ch_host}:{ch_port}/{ch_db}"

In [70]:
df_csv_click.write \
    .format("jdbc") \
    .option("url", f'jdbc:clickhouse://{ch_host}:{ch_port}/{ch_db}') \
    .option("dbtable", ch_table) \
    .option("user", ch_user) \
    .option("password", ch_password) \
    .option("driver", "com.clickhouse.jdbc.ClickHouseDriver") \
    .mode("append") \
    .save()


25/11/09 14:49:04 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/11/09 14:49:04 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction is not supported. You may change jdbcCompliant to false to throw SQLException instead.
25/11/09 14:49:04 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction [fb534fb0-53d9-43ab-b53c-e788054d1497] (11 queries & 0 savepoints) is committed.
25/11/09 14:49:04 WARN ClickHouseConnectionImpl: [JDBC Compliant Mode] Transaction [665572f4-4368-4301-9c48-2b80b0a49490] (0 queries & 0 savepoints) is committed.


In [76]:
visits = spark.read.format("jdbc") \
        .option('url', f'jdbc:clickhouse://{ch_host}:{ch_port}/{ch_db}') \
        .option("dbtable", ch_table) \
        .option("user", ch_user) \
        .option("password", ch_password) \
        .option("driver", "com.clickhouse.jdbc.ClickHouseDriver") \
        .load()

visits.show(8)

+-------+-------------------+--------------------+--------+--------+--------+--------------------+----------------+
|visitid|      visitDateTime|                 URL|duration|clientID|  source|         UTMCampaign|          params|
+-------+-------------------+--------------------+--------+--------+--------+--------------------+----------------+
| 100059|2024-08-31 14:08:00|https://our-cool-...|      79|     522|  direct|month_openconcept...|              []|
| 100094|2024-06-09 09:30:12|https://our-cool-...|      20|     847|internal|month_contemporar...|              []|
| 100109|2024-04-03 17:07:17|https://our-cool-...|      66|     121|  direct|year_traditional_...|['submit', 4315]|
| 100150|2024-02-02 18:21:08|https://our-cool-...|      12|     790|  direct|month_contemporar...|              []|
| 100164|2024-07-13 15:56:27|https://our-cool-...|      44|     958|  direct|year_luxury_kitch...|['submit', 7580]|
| 100230|2024-11-05 15:09:03|https://our-cool-...|      69|     861|inte

In [75]:
visits.show(1, truncate=False, vertical=True)

-RECORD 0------------------------------------------------------
 visitid       | 100059                                        
 visitDateTime | 2024-08-31 14:08:00                           
 URL           | https://our-cool-website.com/featured         
 duration      | 79                                            
 clientID      | 522                                           
 source        | direct                                        
 UTMCampaign   | month_openconcept_kitchens_promotion_20240628 
 params        | []                                            
only showing top 1 row


In [77]:
visits.printSchema()

root
 |-- visitid: integer (nullable = true)
 |-- visitDateTime: timestamp (nullable = true)
 |-- URL: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- clientID: integer (nullable = true)
 |-- source: string (nullable = true)
 |-- UTMCampaign: string (nullable = true)
 |-- params: string (nullable = true)



## Готовим источники